In [ ]:
## EDA for Sales ##

### Import Packages ###
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib as plt
import us
from uszipcode import SearchEngine

# Set Zipcode Search Engine
search = SearchEngine(simple_zipcode=True)

# Default Style for Seaborn
sns.set_theme(style="whitegrid")

In [ ]:
## Load Dataset ##
sales = pd.read_csv("Sales_2020_ANONYMIZED.csv")
sales['date'].astype('datetime64[ns, US/Eastern]')
sales = sales.drop(sales.columns[0], axis=1)
sales['state'] = ''
sales['city'] = ''

## Bring in State/City Data based on Zip
for idx, row in sales.iterrows():
    # Search on zipcode - gets first 5 chars
    zipcode = search.by_zipcode(str(sales.loc[idx, 'name_zip'])[:5])
    # Add to df 
    sales.loc[idx, 'state'] = str(us.states.lookup(str(zipcode.state)))
    sales.loc[idx, 'city'] = zipcode.major_city

In [ ]:
### SPLIT DF ###
# Individual Sales
individual_sales = sales[sales['cust_type'] == "individual"]

# Business Sales
business_sales = sales[sales['cust_type'] == "business"]

# Walk-In Sales
walkin_sales = sales[sales['cust_type'] == "walk-in"]

In [ ]:
## Group Customers ##
transactions_grouped = sales.groupby(['num', 'cust_id'])["amount"].agg(['sum'])
sales_cust_grouped = transactions_grouped.groupby('cust_id')['sum'].agg(['sum', 'count'])

In [ ]:
## Create Summary CSV ##

## Total Sales 
# The sum of 'amount' 
total_sales = round(sales['amount'].sum(), 2)

## Unique Products 
# get the number of unique 'item's
product_qty = sales.groupby('item')['qty'].mean()
unique_prod = round(product_qty.shape[0], 2)

## Unique Customers 
# Count the number of unique 'cust_id'
customer_count = sales.groupby('cust_id')['cust_id'].nunique()
customer_count = round(customer_count.shape[0], 2)

## Avg Amnt per T/A 
# Get the average transaction amount - first aggregated by customers and date to group item purchases on the same day by the same customer (returns the number of T/As) then divided by the total sales
avg_transaction = round(sales['amount'].sum()/transactions_grouped['sum'].count(), 2)

## States Sold To
states_soldto = sales.groupby('state')['state'].nunique()
states_soldto = round(states_soldto.shape[0], 2)

## Cities Sold To
city_soldto = sales.groupby('city')['city'].nunique()
city_soldto = round(city_soldto.shape[0],2)

## Create summary df
# initialize list of lists 
data = [['total sales', total_sales], ['unique products', unique_prod], ['unique customers', customer_count], 
        ['average T/A amount', avg_transaction], ['count states sold to', states_soldto], 
        ['count of cities sold to', city_soldto],
       ]

# Create df 
summary_df = pd.DataFrame(data, columns = ['summary', 'value'])

In [ ]:
## Group by Month/Year ##
sales['YearMonth'] = pd.to_datetime(sales['date']).apply(lambda x: '{year}-{month}'.format(year=x.year,month=x.month))
monthly_sales = sales.groupby('YearMonth').agg({'amount': 'sum', 
                                                'cust_id': 'nunique'})
monthly_sales.sort_values(by="YearMonth")

In [ ]:
## State Summary ##
state_grouped = sales.groupby(['cust_id', 'date', 'state'])["amount"].agg(['sum'])
state_grouped = state_grouped.groupby('state')['sum'].agg(['sum', 'count'])

In [ ]:
## Customer Breakdown ##

# Group sales by month and type
monthly_grouped_sales = sales.groupby(["num","YearMonth", "cust_type"])["amount"].agg(["sum"]).reset_index()
monthly_grouped_sales = monthly_grouped_sales.groupby(["YearMonth", "cust_type"], as_index=False)["num", "sum"].agg(["count", "sum"])

In [ ]:
## Output to CSV ##
monthly_grouped_sales.to_csv("customer_type_monthly_sales.csv")
state_grouped.to_csv('state_grouped.csv')
monthly_sales.to_csv('monthly_sales.csv')
summary_df.to_csv("summary_metrics.csv", index = False)
sales.to_csv("total_sales.csv", index = False)
individual_sales.to_csv("individual_sales.csv", index = False)
business_sales.to_csv("business_sales.csv", index = False)
walkin_sales.to_csv("walk-in_sales.csv", index = False)
transactions_grouped.to_csv("transactionsgrouped.csv")
sales_cust_grouped.to_csv("sales_cust_grouped.csv")